In [1]:
import pandas as pd
import json
import torch
import numpy as np

/opt/anaconda3/envs/llm_env/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
def get_data(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

In [3]:
def format_prompt(entry):
    instruction_text = (
        f"Below is an instruction that describe a task."
        f"Write a response that appropriately completes the request."
        f"\n### Instruction:\n{entry['instruction']}"
    )
    input_text = f"\n### Input:\n{entry['input']}" if entry['input'] else ""
    return instruction_text + input_text

In [4]:
data = get_data('instruction-data.json')
print("Number of Entries:", len(data))

Number of Entries: 1100


In [5]:
train_portion = int(len(data) * 0.85)
test_portion = int(len(data) * 0.15)

train_data = data[:train_portion]
test_data = data[test_portion:]

with open("train.json","w") as json_file:
    json.dump(train_data, json_file, indent=4)
    
with open("test.json","w") as json_file:
    json.dump(test_data, json_file, indent=4)

In [ ]:
!litgpt finetune_lora microsoft/phi-2 \
    --data JSON \
    --data.val_split_fraction 0.1 \
    --data.json_path train.json \
    --train.epoch 3 \
    --train.log_interval 100

In [ ]:
from litgpt import LLM
llm = LLM.load("out/finetune/lora/final")

In [ ]:
from tqdm import tqdm
for i in range(len(test_data)):
    response = llm.generate(test_data[i])
    test_data[i]["response"] = response